---
title: Parker Solar Probe (PSP)
---

In [11]:
# | default_exp missions/psp/__init__
# | export
import polars as pl
from space_analysis.io.cdf import cdf2pl
import pyspedas

from functools import partial
from typing import Literal, overload

# from pydantic import BaseModel

In [1]:

# class InstrumentConfig(BaseModel):
#     default_datatype: str
#     default_var_names: list[str]
    
# configurations: dict[str, InstrumentConfig] = {
#     "fields": InstrumentConfig("mag_rtn", ["psp_fld_l2_mag_RTN"]),
#     "spi": InstrumentConfig("sf00_l3_mom", ["DENS", "VEL_RTN_SUN", "TEMP"]),
# }

In [5]:
class Variable(BaseModel):
    name: str

class Dataset(BaseModel):
    name: str
    variables: dict[str, Variable] = None

    def add_variable(self, variable: Variable):
        self.variables[variable.name] = variable

class Instrument(BaseModel):
    name: str
    datasets: dict[str, Dataset] = None

    def add_dataset(self, dataset: Dataset):
        self.datasets[dataset.name] = dataset

class Mission(BaseModel):
    name: str
    instruments: dict[str, Instrument] = None
    datasets: dict[str, Dataset] = None

    def add_instrument(self, instrument: Instrument):
        self.instruments[instrument.name] = instrument

In [6]:
Mission(
    name="psp"
)

Mission(name='psp', instruments=None, datasets=None)

In [2]:
# | export
def download_data(**kwargs):
    return pyspedas.psp.load(downloadonly=True, **kwargs)

In [9]:
# | export
Instruments = Literal["fields", "spc", "spi"]

MagDataVars = Literal[
    "psp_fld_l2_mag_RTN",
]

SPIDataVars = Literal[
    "QUALITY_FLAG",
    "DENS",
    "VEL_INST",
    "VEL_SC",
    "VEL_RTN_SUN",
    "T_TENSOR_INST",
    "TEMP", # Temperature (eV)
    "EFLUX_VS_ENERGY",
    "EFLUX_VS_THETA",
    "EFLUX_VS_PHI",
    "SUN_DIST",
    "VENUS_DIST",
    "SC_VEL_RTN_SUN",
    "QUAT_SC_TO_RTN",
    "MAGF_SC", # Magnetic Field in Spacecraft Coordinates
    "MAGF_INST",
]

In [14]:
# | export
@overload
def load(
    instrument: Literal["fields"],
    var_names: list[MagDataVars],
    datatype: Literal["mag_rtn"],
    level: Literal["l2"],
    **kwargs
): ...


@overload
def load(
    instrument: Literal["spi"],
    var_names: list[SPIDataVars],
    datatype: Literal["sf00_l3_mom"],
    level: Literal["l3"],
    **kwargs
): ...


def validate(instrument, datatype=None, var_names=None, **kwargs):
    "validate the input arguments and return the correct ones"
    if instrument == "fields":
        datatype = datatype or "mag_rtn"
        var_names = var_names or ["psp_fld_l2_mag_RTN"]
    elif instrument == "spi":
        datatype = datatype or "sf00_l3_mom"
        var_names = var_names or ["DENS", "VEL_RTN_SUN", "TEMP"]
    return var_names, datatype


def load(instrument="fields", var_names=None, datatype=None, **kwargs) -> pl.LazyFrame:

    var_names, datatype = validate(instrument, datatype, var_names, **kwargs)

    files = download_data(instrument=instrument, datatype=datatype, **kwargs)

    load_func = partial(cdf2pl, var_names=var_names)

    return pl.concat(load_func(file) for file in files)